In [2]:
import pandas as pd 
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

In [3]:
data= pd.read_csv("D:\PFE\Fact_Table_Rev.csv", low_memory=False)

In [4]:
data

,montant_HT,montant_ttc,date_saisie,date_facture,Fk_prestation,Fk_famille_prestation,Fk_nationnalite,Fk_Societe,Fk_typesociete
0,4.0,3.571429,2006-09-18,2006-09-18,BOI,BAR,TN,STANDART,SOC
1,6.0,5.357143,2006-09-18,2006-09-18,BOI,BAR,TN,STANDART,SOC
2,8.0,7.142857,2006-09-18,2006-09-18,BOI,BAR,TN,STANDART,SOC
3,10.0,8.928571,2006-09-18,2006-09-18,BOI,BAR,TN,STANDART,SOC
4,8.5,7.589286,2006-09-18,2006-09-18,BOI,BAR,TN,STANDART,SOC
...,...,...,...,...,...,...,...,...,...
937562,10.5,9.375000,2013-09-24,2013-09-24,BOI,BAR,LI,GRINPARAPH,LAB
937563,7.0,6.250000,2013-09-24,2013-09-24,BOI,BAR,LI,GRINPARAPH,LAB
937564,3.5,3.125000,2013-09-24,2013-09-24,BOIR,CAVE,TN,STANDART,SOC
937565,196.0,175.000000,2013-09-24,2013-09-24,REP,RESTO,TN,STANDART,SOC


In [5]:
df = pd.DataFrame(data, columns=["montant_HT", "date_saisie", "Fk_famille_prestation","Fk_typesociete"])

In [6]:
df['date_saisie'] = pd.to_datetime(df['date_saisie'])

In [7]:
df

,montant_HT,date_saisie,Fk_famille_prestation,Fk_typesociete
0,4.0,2006-09-18,BAR,SOC
1,6.0,2006-09-18,BAR,SOC
2,8.0,2006-09-18,BAR,SOC
3,10.0,2006-09-18,BAR,SOC
4,8.5,2006-09-18,BAR,SOC
...,...,...,...,...
937562,10.5,2013-09-24,BAR,LAB
937563,7.0,2013-09-24,BAR,LAB
937564,3.5,2013-09-24,CAVE,SOC
937565,196.0,2013-09-24,RESTO,SOC


In [8]:
df= df.drop(index=df.loc[(df['date_saisie'] < '2006-01-01') | (df['date_saisie'] > '2019-12-31')].index)

In [9]:
df= df.sort_values(by='date_saisie', ascending=True)

In [9]:
print(df)

        montant_HT date_saisie Fk_famille_prestation Fk_typesociete
0              4.0  2006-09-18                   BAR            SOC
313            0.0  2006-09-18                   BAR            SOC
314            0.0  2006-09-18                   BAR            SOC
315            0.0  2006-09-18                   BAR            SOC
316            0.0  2006-09-18                   BAR            SOC
...            ...         ...                   ...            ...
765368       120.0  2019-12-31                TORRET            SOC
765367       147.0  2019-12-31                TORRET            SOC
765366        16.0  2019-12-31                TORRET            SOC
765364        44.0  2019-12-31                TORRET            SOC
765337        12.0  2019-12-31                TORRET            SOC

[753932 rows x 4 columns]


In [10]:
label_encoder = LabelEncoder()
df['Fk_prestation_code'] = label_encoder.fit_transform(df['Fk_famille_prestation'])
label_encoder = LabelEncoder()
df['Fk_typesociete_code'] = label_encoder.fit_transform(df['Fk_typesociete'])
df

,montant_HT,date_saisie,Fk_famille_prestation,Fk_typesociete,Fk_prestation_code,Fk_typesociete_code
0,4.0,2006-09-18,BAR,SOC,2,5
313,0.0,2006-09-18,BAR,SOC,2,5
314,0.0,2006-09-18,BAR,SOC,2,5
315,0.0,2006-09-18,BAR,SOC,2,5
316,0.0,2006-09-18,BAR,SOC,2,5
...,...,...,...,...,...,...
765368,120.0,2019-12-31,TORRET,SOC,19,5
765367,147.0,2019-12-31,TORRET,SOC,19,5
765366,16.0,2019-12-31,TORRET,SOC,19,5
765364,44.0,2019-12-31,TORRET,SOC,19,5


In [11]:
df1 = df[['Fk_typesociete', 'Fk_typesociete_code']].drop_duplicates()
print(df1)

      Fk_typesociete  Fk_typesociete_code
0                SOC                    5
909              PET                    4
750              AGC                    0
6368             LAB                    2
14020            BAN                    1
54249            OPE                    3


Random forest


In [13]:
X_train = df[(df['date_saisie'].dt.year >= 2006) & (df['date_saisie'].dt.year <= 2018)][['Fk_typesociete_code']]
X_test = df[df['date_saisie'].dt.year == 2019][['Fk_typesociete_code']]
y_train = df[(df['date_saisie'].dt.year >= 2006) & (df['date_saisie'].dt.year <= 2018)]['montant_HT']
y_test = df[df['date_saisie'].dt.year == 2019]['montant_HT']

In [33]:
rf = RandomForestRegressor()
rf.fit(X_train, y_train)

RandomForestRegressor()

In [34]:
y_pred = rf.predict(X_test)

In [35]:
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 138347.12797877984


In [36]:
y_pred.sum()

3799547.9112890884

In [37]:
result_df = pd.DataFrame({'y_pred': y_pred, 'Fk_typesociete_code': X_test['Fk_typesociete_code']})
sum_by_prestation_code = result_df.groupby('Fk_typesociete_code')['y_pred'].sum()
mapping_df = df[['Fk_typesociete_code', 'Fk_typesociete']].drop_duplicates()

# Merge total_montant_HT_by_code with mapping_df on Fk_typesociete_code
Predtypesociete= sum_by_prestation_code.reset_index().merge(mapping_df, on='Fk_typesociete_code')

# Display the result
print(Predtypesociete)

   Fk_typesociete_code        y_pred Fk_typesociete
0                    0  1.713926e+05            AGC
1                    1  2.616393e+04            BAN
2                    2  2.884122e+05            LAB
3                    3  2.094426e+03            OPE
4                    4  5.390195e+05            PET
5                    5  2.772465e+06            SOC


In [38]:
Predtypesociete.to_csv('Predtypesociete.csv', index=False)

Sarimax


In [12]:
selected_columns = ['date_saisie', 'montant_HT','Fk_typesociete']
df1 = df.loc[:, selected_columns]
df1['year'] = df['date_saisie'].dt.year.astype(int)
df1['month'] = df['date_saisie'].dt.month.astype(int)
df1['day'] = df['date_saisie'].dt.day.astype(int)
selected_columns = ['date_saisie','year','month', 'day','montant_HT','Fk_typesociete']
df1 = df1.loc[:, selected_columns]

In [13]:
df1

,date_saisie,year,month,day,montant_HT,Fk_typesociete
0,2006-09-18,2006,9,18,4.0,SOC
313,2006-09-18,2006,9,18,0.0,SOC
314,2006-09-18,2006,9,18,0.0,SOC
315,2006-09-18,2006,9,18,0.0,SOC
316,2006-09-18,2006,9,18,0.0,SOC
...,...,...,...,...,...,...
765368,2019-12-31,2019,12,31,120.0,SOC
765367,2019-12-31,2019,12,31,147.0,SOC
765366,2019-12-31,2019,12,31,16.0,SOC
765364,2019-12-31,2019,12,31,44.0,SOC


In [14]:
df1= df1.groupby([ 'date_saisie','month', 'Fk_typesociete'])['montant_HT'].sum().reset_index()

In [15]:
df1.set_index('date_saisie', inplace=True)

In [16]:
df1

,month,Fk_typesociete,montant_HT
date_saisie,,,
2006-09-18,9,SOC,111.00
2006-09-19,9,PET,80.00
2006-09-19,9,SOC,183.00
2006-09-20,9,SOC,1133.50
2006-09-21,9,SOC,1756.50
...,...,...,...
2019-12-30,12,AGC,64.00
2019-12-30,12,PET,1412.40
2019-12-30,12,SOC,5373.35


In [17]:
index_type = type(df1.index)

# Check if it's a DateTimeIndex
is_datetime_index = isinstance(df1.index, pd.DatetimeIndex)

# Print the result
print("Index type:", index_type)
print("Is DateTimeIndex:", is_datetime_index)

Index type: <class 'pandas.core.indexes.datetimes.DatetimeIndex'>
Is DateTimeIndex: True


In [18]:
train_data = df1.loc[:'2018']
test_data = df1.loc['2019':]

In [19]:
from tqdm import tqdm 
import warnings
warnings.filterwarnings("ignore")


results = list()
all_prestation = df1.Fk_typesociete.unique()
for i in tqdm(range(len(all_prestation))):
    prestation = all_prestation[i]
    sub_df = df1[df1.Fk_typesociete == prestation]
    train_data = sub_df.loc[:'2018']
    test_data = sub_df.loc['2019':]
    model = SARIMAX(train_data['montant_HT'],  order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    result = model.fit()
    start = len(train_data)
    end = len(train_data) + len(test_data) -1
    if start > end:
        print('start is greater than end')
        continue
    else:
        predictions = list(result.predict(start, end))
        prestation_result = pd.DataFrame()
        prestation_result["montant_predit"] = predictions
        prestation_result["date"] = list(test_data.index)
        prestation_result["Fk_typesociete"] = prestation
        results.append(prestation_result)

100%|██████████| 6/6 [00:47<00:00,  7.99s/it]


In [20]:
final_pred = pd.concat(results)

In [21]:
test_data = df1.loc['2019':].reset_index()
test_data.rename(columns= {"date_saisie":'date'},inplace=True)

In [22]:
merged_data=pd.merge(left = test_data, right = final_pred, how = 'left', on = ['date','Fk_typesociete'])

In [24]:
sum_by_code = merged_data.groupby('Fk_typesociete').agg({
    'montant_HT': 'sum',
    'montant_predit': 'sum'
}).reset_index()

print(sum_by_code)

  Fk_typesociete    montant_HT  montant_predit
0            AGC  2.637763e+05    2.111736e+05
1            BAN  7.413500e+04    1.062133e+04
2            LAB  4.136254e+05    3.422407e+05
3            OPE  1.716000e+03    1.244611e+03
4            PET  7.596027e+05    4.875873e+05
5            SOC  3.341769e+06    4.606928e+06
